The steps are mostly based on these refrences:
- [Fine-tuning XLS-R for Multi-Lingual ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-xlsr-wav2vec2)
- [Fine_Tune_XLSR_Wav2Vec2_on_Persian_ShEMO](https://github.com/m3hrdadfi/notebooks/blob/main/Fine_Tune_XLSR_Wav2Vec2_on_Persian_ShEMO_ASR_with_%F0%9F%A4%97_Transformers_ipynb.ipynb)

You can consult them if you want to see a more detailed procedure.

The second one also contains valuable hints on how to preprocess the persian text for our purpose.

In [ ]:
# Download nessecary libraries
!pip install datasets==2.10.0 --quiet
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.0 MB/s eta 0:00:00


## Loading the dataset

In [ ]:
# If you wish your data to persist even when you shutdown colab, save your intermediate results to your google drive
# Then in the code you can change the saving path of files to ./drive/MyDrive/ path, which is your google drive disk
from os import path,system,mkdir
from google.colab import drive

drive.mount('/content/drive/')
if not path.exists('./drive/MyDrive/ASR_Colab'):
  mkdir('./drive/MyDrive/ASR_Colab')

Mounted at /content/drive/


In [ ]:
# modify the path to dataset if nessecary
dataet_path = './drive/MyDrive/ML_Project_ASR/trimmed_dataset.zip'
if not path.exists('dataset'):
  system(f'unzip -qq "{dataet_path}" -d "/content/"')

In [ ]:
# Load csv
# We use pandas for data import, and datasets lib to prepear our data.
# These two libs (pandas.Dataframe & datasets.Dataset) are convertable as shown bellow, so use whichever you find more convenient
import pandas as pd
from datasets import Dataset

transcripts = pd.read_csv('dataset/clean_dataset.csv')
ds = Dataset.from_pandas(transcripts)
transcripts.head()

voice_filename                                         transcript accent  \
0    voice_1.mp3                      چرا این‌‌‌‌طور فکر می‌‌‌‌کنی؟  فارسی   
1    voice_2.mp3    همیشه من و تو راجع به آن با هم صحبت کرده‌‌‌‌ایم  فارسی   
2    voice_3.mp3             دنیا در حال گذار به‌‌‌‌سمت پایداری است  فارسی   
3    voice_4.mp3   شاخصی که باید عملکرد تسلا را با آن اندازه بگیریم  فارسی   
4    voice_5.mp3  باید تعداد واقعاً غیرقابل‌‌‌‌تصوری باتری تولید...  فارسی   

  gender      tone  
0   male  question  
1   male    normal  
2   male    normal  
3   male    normal  
4   male    normal

In [ ]:
# Take a look at unique letters in our dataset
from functools import reduce
present_chars = reduce(lambda a, b: set((*a,*b)), list(ds['transcript']))
print(f'[{len(present_chars)}]=>',present_chars)

[163]=> {'م', ':', '\t', 'ﻒ', 'ٍ', '9', 'ا', '٪', 'ﺕ', '۵', '2', 'ﻝ', 'ﺮ', 'ﺖ', 'ﭽ', 'ﻨ', ')', '\u202b', '!', 'ﺪ', '۲', '0', 'ﺷ', 'ظ', 'ﺛ', 'ﻌ', 'ﮐ', '6', 'ؤ', 'ّ', '\xad', 'ﻏ', 'ﺥ', '»', 'M', 'ل', 'ء', 'گ', 'ْ', '5', 'ﺤ', 'ﻡ', 'ﯾ', 'ﻭ', 'ﺟ', 'و', 'ى', 'ﻃ', 'ـ', '\u200c', '…', 'ﻥ', '٬', 'ﺍ', '۸', 'ﺶ', 'د', 'ﺗ', 'س', 'ﻖ', 'ﻣ', ',', 'ﺁ', '(', '۴', '7', 'ﺨ', '؟', '۳', 'ث', 'ﺼ', 'ﻩ', 'ۀ', 'ع', 'ك', 'ﻤ', 'ی', 'ص', '3', '-', ' ', 'ئ', 'V', 'ﻬ', 'ﻫ', 'ﺑ', 'ﺲ', 'غ', 'ف', '۶', 'ﻠ', 'ﮔ', 'ﺘ', 'ﺴ', '٫', 'S', 'ش', '”', '۱', '“', 'ب', 'ق', 'ج', 'ز', 'ﻪ', '–', '\n', 'ض', 'ن', 'ﻦ', 'ح', '4', 'خ', '\u202c', '/', 'ﻔ', 'ه', '،', 'ﻢ', 'ر', 'ک', '.', 'ﺳ', '۹', '1', 'ٔ', 'ﺫ', '؛', '\xa0', 'ت', 'ِ', '"', 'پ', 'ﮑ', 'ط', 'ﻮ', 'ﻧ', '۷', '۰', 'ﺩ', 'ُ', 'ﻓ', 'ﺣ', '٥', 'آ', 'ً', 'ﺎ', 'ﮏ', 'ذ', 'ﯽ', 'َ', 'ﭘ', '8', 'چ', '«', 'ﺭ', 'أ', '٨', 'ﯼ', 'ﯿ', 'ژ', 'ي', 'ﺯ'}


## Preprocessing text & audio

* Normalize texts

In [ ]:
!pip install hazm

import hazm

In [ ]:
normalizer = hazm.Normalizer()
transcripts['transcript'] = transcripts['transcript'].apply(lambda x: normalizer.normalize(x))
ds = Dataset.from_pandas(transcripts)

In [ ]:
# Take a look at unique letters in our dataset
from functools import reduce
present_chars = reduce(lambda a, b: set((*a,*b)), list(ds['transcript']))
print(f'[{len(present_chars)}]=>',present_chars)

[80]=> {'م', ':', 'ا', '٪', '۵', ')', '\u202b', '!', '۲', 'ظ', 'ؤ', '\xad', '»', 'M', 'ل', 'ء', 'گ', 'ﯾ', 'و', '\u200c', '…', '٬', '۸', 'د', 'س', ',', '(', '۴', '؟', '۳', 'ث', 'ۀ', 'ع', 'ی', 'ص', '-', ' ', 'ئ', 'V', 'غ', 'ف', '۶', '٫', 'S', 'ش', '۱', 'ب', 'ق', 'ج', 'ز', '–', '\n', 'ض', 'ن', 'ح', 'خ', '\u202c', '/', 'ه', '،', 'ر', 'ک', '.', '۹', '؛', 'ت', '"', 'پ', 'ط', '۷', '۰', 'آ', 'ذ', 'ﯽ', 'چ', '«', 'أ', 'ﯼ', 'ﯿ', 'ژ'}


In [ ]:
print(transcripts['transcript'][4569])
transcripts['transcript'][4569] = transcripts['transcript'][4569].replace('٪','درصد')
print(transcripts['transcript'][4569])

ایا می‌توانید تصور کنید که اقیانوس‌ها ۹۷ ٪ از حجم زمین را به خود اختصاص داده‌اند
ایا می‌توانید تصور کنید که اقیانوس‌ها ۹۷ درصد از حجم زمین را به خود اختصاص داده‌اند


In [ ]:
transcripts.to_csv('/content/drive/MyDrive/ML_Project_ASR/cleaned_dataset.csv',encoding='utf-8')

In [ ]:
# Some of the listed chars are the same, but have different representations(like 'ب' & 'ﺑ')
# They should get combined(one of them gets mapped to the other)
# complete the following dict:
import re

char_mappings = {'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
                'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
                "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
                "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
                'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
                'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",
                # My dicts:
                'ئ': 'ی',
                '0': '۰', '1': '۱', '2': '۲', '3': '۳', '4': '۴',
                '5': '۵', '6': '۶', '7': '۷', '8': '۸', '9': '۹',
}

def multiple_replace(batch, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    batch['transcript'] = re.sub(pattern, lambda m: chars_to_mapping[m.group()], batch['transcript'])
    return batch

ds = ds.map(lambda batch: multiple_replace(batch,char_mappings))

Map:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
present_chars = reduce(lambda a, b: set((*a,*b)), list(ds['transcript']))
print(f'[{len(present_chars)}]=>',present_chars)

[73]=> {'ک', 'م', 'گ', 'V', '.', ':', 'و', 'غ', '۹', 'ف', '۶', 'ا', '؛', 'ت', '٪', '"', '\u200c', 'پ', '…', '٫', 'ط', '۵', '٬', 'S', '۸', 'ش', 'د', '۷', 'س', '۱', '۰', ')', ',', 'ب', 'ق', '!', '\u202b', 'ج', '۲', '(', 'آ', 'ز', '۴', 'ظ', '–', '\n', 'ذ', '؟', 'ض', '۳', 'ث', 'ء', 'ن', 'چ', 'ح', '«', 'ع', '\xad', 'خ', '»', '\u202c', 'M', 'ی', 'ص', 'ل', 'ﯼ', '/', 'ه', 'ژ', '-', '،', 'ر', ' '}


In [ ]:
# Some chars don't have any sound, so they should get removed
# Don't remove the ' ' (space) though, as the model should learn to predict when each word ends
# handle the transcripts containing numbers as you deem nessecary
# complete the following list:
import string

char_removals = [
    '\u200c','(',')','!','،','\u202c','«','»','…','ٍ','ـ',',', ':', '.',
    '\t','\n', '٪', '”', '؛', '"', '/',
    "ء",      # hamze
    "\u064E", # Fatha
    "\u064B", # Fathatan
    "\u064F", # Damma
    "\u08F1", # Dammatan
    "\u061A", # Kasra
    "\u064D", # Kasratan
    "\u0651", # Tashdid
    "\u0652", # Sokun
    ] + list(string.ascii_letters + string.digits)
def remove_special_characters(batch,char_removals):
    chars_to_ignore_regex = f"""[{"".join(char_removals)}]"""
    batch['transcript'] = re.sub(chars_to_ignore_regex, '', batch['transcript']).lower() + " "
    return batch

ds = ds.map(lambda batch: remove_special_characters(batch,char_removals))

Map:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
# The resulting vocab(list of letters):
vocab = reduce(lambda a, b: set((*a,*b)), ds['transcript'])
print(f'[{len(vocab)}]=>',vocab)

[52]=> {'ک', 'م', 'گ', 'و', 'غ', '۹', 'ف', '۶', 'ا', 'ت', 'پ', '٫', 'ط', '۵', '٬', '۸', 'ش', 'د', '۷', 'س', '۱', '۰', '\u202b', 'ب', 'ق', 'ج', '۲', 'آ', 'ز', '۴', 'ظ', '–', 'ذ', '؟', 'ض', '۳', 'ث', 'ن', 'چ', 'ح', 'ع', '\xad', 'خ', 'ی', 'ص', 'ل', 'ﯼ', '-', 'ه', 'ژ', 'ر', ' '}


In [ ]:
# Wav2Vec requires some special tokens to be added to vocab
# We also replace ' '(space) with '|' for more visibility
# The vocab should get saved as a json file and later get used by the model
vocab_dict = {v: k for k, v in enumerate(vocab)}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

import json
with open('./drive/MyDrive/ASR_Colab/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

To know what is the role of tokenizer, feature extractor, data collator & etc. in this model, visit https://huggingface.co/blog/fine-tune-xlsr-wav2vec2

In [ ]:
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2FeatureExtractor,Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("./drive/MyDrive/ASR_Colab/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
# Tokenizing the transcripts and then load,convert to mono channel and resample audio files at 16 KHz
import librosa
import warnings

def prepare_dataset(batch):
  file_path = path.join('dataset','trimmed_voices',batch['voice_filename'])
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    speech_array, sampling_rate = librosa.load(file_path,mono=True,sr=16000)

    batch["input_values"] = processor(speech_array, sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"]).input_ids

  return batch

ds = ds.map(prepare_dataset)

Map:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
# To reduce GPU memory usage, filter out voice samples that are too long:
max_input_length_in_sec = 15
ds = ds.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
ds = ds.train_test_split(0.2)

# A report on dataset length:
ds

DatasetDict({
    train: Dataset({
        features: ['voice_filename', 'transcript', 'accent', 'gender', 'tone', 'input_values', 'input_length', 'labels'],
        num_rows: 4680
    })
    test: Dataset({
        features: ['voice_filename', 'transcript', 'accent', 'gender', 'tone', 'input_values', 'input_length', 'labels'],
        num_rows: 1171
    })
})

In [ ]:
# Save for later use
ds.save_to_disk("./drive/MyDrive/ASR_Colab/dataset.hf")

Flattening the indices:   0%|          | 0/4680 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/4680 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1171 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1171 [00:00<?, ? examples/s]